In [11]:
import csv
import cv2
import numpy as np

In [2]:
#read csv data file
lines = []

with open('./data/driving_log.csv') as csvDataFile:
    csvReader = csv.reader(csvDataFile)
    for row in csvReader:
        lines.append(row)

In [4]:
lines[0]

['/Users/manika/Desktop/data/IMG/center_2017_10_25_18_26_51_303.jpg',
 '/Users/manika/Desktop/data/IMG/left_2017_10_25_18_26_51_303.jpg',
 '/Users/manika/Desktop/data/IMG/right_2017_10_25_18_26_51_303.jpg',
 '0',
 '0',
 '0',
 '9.12E-07']

In [6]:
#read the center images
images = []
measurements = []
for line in lines:
    imagePath = line[0]
    image = cv2.imread(imagePath)
    images.append(image)
    measurement = float (line[3])
    measurements.append(measurement)

In [8]:
print (images[0])

[[[185 145 116]
  [185 145 116]
  [184 144 115]
  ..., 
  [ 46  53  62]
  [  0   0   7]
  [  4  11  20]]

 [[185 145 116]
  [185 145 116]
  [185 145 116]
  ..., 
  [ 24  32  39]
  [ 11  19  26]
  [  3  11  18]]

 [[186 146 117]
  [186 146 117]
  [186 146 117]
  ..., 
  [ 20  27  30]
  [ 21  28  31]
  [ 49  56  59]]

 ..., 
 [[ 84  91  86]
  [ 81  88  83]
  [ 74  81  76]
  ..., 
  [124 140 139]
  [ 91 107 106]
  [ 71  87  86]]

 [[ 75  82  77]
  [ 52  59  54]
  [ 65  72  67]
  ..., 
  [ 94 110 109]
  [ 88 104 103]
  [ 68  84  83]]

 [[ 39  46  41]
  [ 68  75  70]
  [ 56  63  58]
  ..., 
  [ 91 107 106]
  [ 60  76  75]
  [ 74  90  89]]]


In [9]:
measurements[0]

0.0

In [12]:
#convert the images and measurements to train data and label
X_train = np.array(images)
y_train = np.array(measurement)